<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_swnlzv.png" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div> 

<br><br><br><br>
<br>

# Simulation of dose-response curves of antagonists using affinity values

In this tutorial, we will simulate a mathematical model of a signaling pathway to obtain dose-response curves of antagonists, from which the *potencies (IC$_{50}$)* values can be inferred. 


<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial2.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>
</div>  


## Installing the SSB toolkit library

In [ ]:
%%capture
!pip install --quiet ssbtoolkit

### Importing sependencies

In [ ]:
import ssbtoolkit as ssb
import numpy as np
import pandas as pd

If you are using the **Ebrains Platform**, execute the following cell to set up the environment. If you are not using **Ebrains**, skip the following step and proceed with the tutorial.

In [ ]:
import os
os.environ['BNGPATH'] = '/opt/app-root/src/.local/lib/python3.8/site-packages/bionetgen/bng-linux/'

## Loading experimental data

Once the SSBtoolkit environment is set up, we are ready to start the simulation.

We will begin by loading the affinity data for the Adenosine 2A receptor, including data for its natural ligand (adenosine) as well as some antagonists. The experimental values were obtained from the *[Guide to Pharmacology](https://www.guidetopharmacology.org/GRAC/ObjectDisplayForward?objectId=19&familyId=3&familyType=GPCR) website*. This data can be downloaded directly from the repository with the following command:

In [ ]:
!curl -s -O https://raw.githubusercontent.com/rribeiro-sci/SSBtoolkit/main/data/A2AR_antagonists_data.csv

In [ ]:
data = pd.read_csv('A2AR_antagonists_data.csv')
data

In <b>Tutorial1</b>, the affinity values were directly used to calculate the fraction of occupied receptors according to occupancy theory. This approach is valid only for agonists. In this tutorial, however, we will simulate competitive antagonism. By definition, a competitive antagonist inhibits the action of an agonist but has no effect in the absence of the agonist.

To calculate the fraction of occupied receptors by agonists in the presence of an antagonist, we use the following equation:

$\frac{[L_1R]}{{R_{total}}}=\frac{[L_1]}{[L_1]+K_{d_{L1}}\big(1+\frac{[L_2]}{K_{d_{L2}}}\big)}$

In experimental settings, antagonist studies are typically conducted through radioligand binding assays. In these assays, the receptor is first saturated with the agonist, and then aliquots of the antagonist are added to the system to generate an inhibition curve.

To replicate this experiment *in silico*, we need to start by calculating the concentration of the radioligand/agonist (in this case, adenosine) that saturates the receptor. We will use the SSB Toolkit’s built-in function `Binding.Bind()`to obtain a ligand-receptor binding curve. Then, we will use the `SubMaxConcentration()` method to interpolate the submaximal (saturating) concentration value.


📖 For more details, refer to the [The SSB toolkit](https://ssbtoolkit.readthedocs.io/) documentation.


In [ ]:
# Setting the ligand concentration range
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E2 # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20)

# Setting receptor's concentration
receptor_conc = 1E-3 #μM

# Getting adenosine pKi from experimental data
adenosine_pKi = data[data.Compound == 'adenosine'].pKi.item()

In [ ]:
# To obtain the ligand-receptor binding curve for adenosine we will start by initiating a binding instance
adenosine_binding = ssb.Binding()

# Performing the binding calculation
adenosine_binding.Bind(receptor_conc=receptor_conc, lig_conc_range=lig_conc_range, pKd=adenosine_pKi)

#Extraction of the submaximal concentration value
adenosine_binding.SubMaxConcentration()

#Plot of the ligand-receptor binding curve
adenosine_binding.ShowCurve()

The submaximal concentration value can be accessed using the `submax_concentration` attribute

In [ ]:
submax = adenosine_binding.submax_concentration
submax

Once we have the submaximal concentration value of adenosine that saturates the receptor, we can calculate the fraction of receptors occupied by adenosine in the presence of various antagonist concentrations. This calculation is performed automatically during the simulation.

## Selection of the signaling pathway 

The core of the SSB framework is, naturally, the mathematical models of GPCR signaling pathways.

G-protein sub-families are classified by their $\alpha$ subunits, which are used to identify their corresponding signaling pathways:

* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 For more details, refer to the [The SSB toolkit](https://ssbtoolkit.readthedocs.io/) documentation.

You can manually define the G$\alpha$ pathway you want to work with, or use the SSB Toolkit’s built-in function `Utils.GetGProtein()` to query our internal database of human GPCR pathways using the UNIPROT ID of the receptor. The UNIPROT ID for the human Adenosine A2 receptor is [P29274](https://www.uniprot.org/uniprot/P29274).



In [ ]:
uniprotID = 'P29274'
gprotein = ssb.Utils.GetGProtein(uniprotID)
gprotein

<span style="color:black">⚠️ WARNING: The toolkit is specifically designed for human GPCRs. Querying pathways for GPCRs other than human may be included in future updates. However, if you wish to study a non-human GPCR, you can still use the SSB Toolkit by manually setting the G$\alpha$ pathway.</span>


##  Preparation, Simulation and Analysis

To obtain a dose-response curve from the simulation of signaling pathways, you need to perform individual simulations of the pathway across a series of ligand concentrations, similar to what would be done in a wet lab.

We will define an array of ligand concentrations using a geometric progression. This approach is chosen because it addresses the impact of dilution fractions on the accuracy of experimentally estimated K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values. The spacing between adjacent concentration values affects the precision of the measurements, particularly in the linear portion of the curve. By using a geometric progression, we can better simulate experimental conditions where each concentration is a power of 2 of the previous concentration *[Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426)*

<span style="color:black"> ⚠️ WARNING: The SSB toolkit uses μM as default unit for concentration values.</span>

In [ ]:
# the range of ligand concentration
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

# Setting receptor concentration
receptor_conc = 1E-3 #μM

# defining other simulation parameters:
time = 10000  # time of simulation in seconds
nsteps = 1000   # number of time steps

## Integration of ODEs 

Once all the simulation parameters have been defined, we are ready to proceed with the simulations. Simulating a mathematical model of a signaling pathway involves integrating a set of ordinary differential equations (ODEs) over time. Each ODE represents a molecular event within the signaling pathway, and integrating these equations describes how the concentration of species in the model changes over time.

A key aspect of this tutorial is using the drug-receptor affinity value (K<sub>d</sub>) to initiate the model. With K<sub>d</sub> values, we can calculate the fraction of receptors occupied by the ligand at equilibrium. According to *occupancy theory*, this fraction represents the concentration of activated receptors at equilibrium *[Kenakin T., 2004 ](https://doi.org/10.1016/j.tips.2004.02.012)*. 📖 For more details, refer to the [Docs](https://ssbtoolkit.readthedocs.io/).


In this tutorial, we will simulate dose-response curves of antagonists targeting the Adenosine A2 receptor using the SSB Toolkit’s built-in class`Simulation.Inhibition()`. 


<span style='color:black'>ℹ️ If you are interested in studying agonists, follow the tutorial on [Simulation of dose-response curves of agonists using affinity values](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial1.ipynb).</span>


In [ ]:
# we will start by creating a simulation instance.
sim = ssb.Simulation.Inhibition()

In [ ]:
# configuration of simulation parameters
sim.SetSimulationParameters(agonist='adenosine',
                            agonist_affinity=adenosine_pKi,
                            agonist_submaximal_conc=submax,
                            antagonists=data.Compound.to_list()[1:3],
                            antagonists_affinities=data.pKi.to_list()[1:3], 
                            lig_conc_range=lig_conc_range, 
                            receptor_conc=receptor_conc, 
                            pathway=gprotein, 
                            ttotal=time, 
                            nsteps=nsteps, 
                            binding_kinetics=False, 
                            binding_kinetic_parameters=None)

In [ ]:
# Starting the simulation
sim.Run()

At the end of the simulation, the concentration values of the species within the signaling pathway will be saved in the instance.

The response of a signaling pathway is typically represented by the increase or decrease in the concentration of one of the species described by the model. To predict the dose-response curve, we need to first extract the maximum concentration value observed for a particular species from each individual simulation (for each ligand concentration). These maximum values will then be fitted to a logistic regression.

To achieve this, we will use the following analysis function:


In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.ShowCurve()

Finnaly, from the dose-response curves we can interpolate the IC<sub>50</sub> values.


In [ ]:
sim.ShowPotency()

💡 The predicted potency values can be exported as a Python dictionary using the function `sim.PotencyToDict()`, or saved to a CSV file using `sim.PotencyToCSV()`. 

## Congratulations!

Congratulations on completing this tutorial notebook! If you enjoyed working through it and would like to continue exploring the SSB Toolkit, we encourage you to complete the rest of the tutorials in this series.

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{ribeiro_ssb_2022,
    title={Structural Systems Biology Toolkit (SSBtoolkit): From Molecular Structure to Subcellular Signaling Pathways},
    author={Ribeiro, Rui Pedro and Gossen, Jonas and Rossetti, Giulia and Giorgetti, Alejandro},
    publisher={J. Chem. Inf. Model.},
    url={https://doi.org/10.1021/acs.jcim.5c00165},
    year={2025}
}
```


## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  